In [47]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, HBox, VBox, Output, FloatSlider
import ipywidgets as widgets

# Function to update plots
def update_plots(freq):

    # Step 1: Generate sample data
    np.random.seed(0)  # For reproducibility
    x = np.random.rand(100) * 10  # Unobserved variable
    y = 2.5 * x + np.random.randn(100) * 2  # Observed quantity with some noise

    # Step 2: Calculate best fit line
    slope, intercept = np.polyfit(x, y, 1)
    best_fit_y = slope * x + intercept

    # Clear previous outputs
    out1.clear_output(wait=True)
    out2.clear_output(wait=True)
    out3.clear_output(wait=True)
    
    # Plot 1
    with out1:
        fig, ax = plt.subplots()
        # Step 3: Plot data and best fit line
        #plt.figure(figsize=(10, 6))
        plt.scatter(x, y, label='Data Points')
        plt.plot(x, best_fit_y, color='red', label='Best Fit Line')
        plt.xlabel('Unobserved Variable')
        plt.ylabel('Observed Quantity')
        plt.legend()
        plt.title('Observed Quantity by Unobserved Variable')
        plt.show()
    
    # Plot 2
    with out2:
        fig, ax = plt.subplots()
        ax.plot(x, y)
        plt.title('Plot 2')
        plt.show()

    # marginal
    with out3:
        fig, ax = plt.subplots()
        plt.figure(figsize=(10, 2))
        ax.plot(x, y)
        plt.xlabel('Observed Quantity')
        plt.show()

# Create two outputs for the plots
out1 = Output()
out2 = Output()
out3 = Output()

# Create a slider
freq_slider = FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Frequency:')

# Step 1: Stack plot1 and plot3 vertically
vbox_for_plot1_and_plot3 = VBox([out1, out3])

# Step 2: Place the VBox from step 1 next to plot2 horizontally
final_layout = HBox([vbox_for_plot1_and_plot3, out2])

# Display the final layout
display(final_layout)

# Call update_plots function when the value of the slider changes
interactive(update_plots, freq=freq_slider)

interactive(children=(FloatSlider(value=1.0, description='Frequency:', max=5.0, min=0.1), Output()), _dom_clas…